In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# reference:


1. softmax 
    + https://keras.io/getting-started/sequential-model-guide/

2. neural network

    + https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
    + https://nttuan8.com/bai-3-neural-network/
    + https://nttuan8.com/bai-4-backpropagation/

3. bag of word
    + https://scikit-learn.org/stable/modules/feature_extraction.html


In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
x_train = pd.ExcelFile('...input/quora-insincere-questions-classification/quora_train.xlsx')
quora_train = x_train.parse('Sheet1')
x_val = pd.ExcelFile('...input/quora-insincere-questions-classification/quora_val.xlsx')
quora_val = x_val.parse('Sheet1')
# x_test = pd.ExcelFile('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/quora_test.xlsx')
# quora_test = x_test.parse('Sheet1')


In [ ]:
quora_test = pd.read_csv('...input/quora-insincere-questions-classification/quora_test1.csv')

In [ ]:
print(quora_train.shape)
print(quora_val.shape)
print(quora_test.shape)
quora_train.head()

In [ ]:
quora_train = quora_train.replace(np.nan, '')
quora_val = quora_val.replace(np.nan, '')
quora_test = quora_test.replace(np.nan, '')
print(quora_train.shape)
print(quora_val.shape)
print(quora_test.shape)

In [ ]:
quora_train.columns

In [ ]:
sentence_train = list(quora_train.text_norm1)
sentence_val = list(quora_val.text_norm1)
sentence_test = list(quora_test.text_norm1)

In [ ]:
quora_train.target = quora_train.target.astype('int')
quora_val.target = quora_val.target.astype('int')

In [ ]:
quora_train.target.value_counts()

In [ ]:
quora_val.target.value_counts()

In [ ]:
y_train = quora_train.target.values
y_val = quora_val.target.values

In [ ]:
y_train.shape

In [ ]:
del quora_train
del quora_val
del quora_test

# 1. Feature engineering

#1.1 Bag of word

In [ ]:
# create feature vector with Countvectorize by sklearn library
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer_bow = CountVectorizer(min_df=0, lowercase=False)
vectorizer_bow.fit(sentence_train)

In [ ]:
X_train_bow = vectorizer_bow.transform(sentence_train)
X_val_bow = vectorizer_bow.transform(sentence_val)
X_test_bow = vectorizer_bow.transform(sentence_test)

In [ ]:
# import pickle
# with open('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/train_bow_matrix.pkl', 'wb') as f:
#   pickle.dump(X_train_bow, f)
  
# with open('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/val_test_bow_matrix.pkl', 'wb') as f:
#   pickle.dump(X_val_bow, f)

# with open('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/test_bow_matrix.pkl', 'wb') as f:
#   pickle.dump(X_test_bow, f)

In [ ]:
# import pickle
# with open('/content/drive/My Drive/quora-insincere-questions-classification/train_bow_matrix.pkl', 'rb') as f:
#   X_train_bow = pickle.load(f)
  
# with open('/content/drive/My Drive/quora-insincere-questions-classification/val_test_bow_matrix.pkl', 'rb') as f:
#   X_val_bow = pickle.load(f)
  
# with open('/content/drive/My Drive/quora-insincere-questions-classification/test_bow_matrix.pkl', 'rb') as f:
#   X_test_bow = pickle.load(f)

Neural network

In [ ]:
# # activation function (sigmoid)
# def sigmoid(x):
#     return (1/(1 + np.exp(-x)))
# # derivative of activation function (sigmoid)
# def sigmoid_derivative(x):
#     return x*(1-x)

# class NeuralNetwork:
#     def __init__(self, layers, alpha = 0.1):
#         '''
#         input:
#         + layers (list) [n1, n2, n3, ..., n_output]
#             the first layer (input layer) contain n1 neurons (input data demension)
#             the second layer contain n2 neurons
#             the third layer contain n3 neurons
#             the last layers (output layer) contain n_ouput neurons
#         + alpha (float)

#         create
#         + layers
#         + W matrix contain the weight matrix between the layers
#         + b matrix contain the bias vector
#         + alpha
#         '''
#         self.layers = layers
#         # learning rate
#         self.alpha = alpha
#         # the parameters matrix W, b
#         self.W = []
#         self.b = []
#         # create the parameters for the each layer
#         for  i in range(0, len(layers) - 1):
#             # create matrix w , that is used connecting layer i and layer i+1. 
#             #the size is (the neurons number of layer i) x (the neurons number of layer i+1)
#             w_ = np.random.rand(layers[i], layers[i+1])
#             # create vector b for the layer i+1
#             # the elements is the neurons number of layer i+1
#             b_ = np.zeros((layers[i+1], 1))
#             self.W.append(w_)
#             self.b.append(b_)
#     # description neural model
#     def __repr__(self):
#         return "Neural network [{}]".format("-".join(str(l) for l in self.layers))
    
#     # train model with data
#     def fit_partial(self, x, y):
#         A = [x]
        
#         # feedforward
#         out = A[-1]
#         for i in range(0, len(self.layers)-1):
#             out = sigmoid(np.dot(out, self.W[i]) + (self.b[i].T))
#             A.append(out)
            
#         # backpropagation
#         y = y.reshape(-1, 1)
#             # init compute derivative of loss function follow y_preditc (A): 
#             # dL/d(y_predict)
#         dA = [-(y/A[-1] - (1-y)/(1-A[-1]))]
#         dW = []
#         db = []
#             # in the step backpropagation
#             # we find W, b follow the result predict from output layer back to input layer
#         for i in reversed(range(0, len(self.layers)- 1)):
#             # dJ/d(W^l) = (A^{l-1}).T * (Y_predict - Y)
#             dw_ = np.dot((A[i]).T, dA[-1]*sigmoid_derivative(A[i+1]))
#             # dJ/d(b^l) = (sum(Y_predict - Y)).T
#             db_ = (np.sum(dA[-1]*sigmoid_derivative(A[i+1]), 0)).reshape(-1, 1)
#             # dJ/d(A^{l-1}) = (Y_predict - Y)*(W^l).T
#             dA_ = np.dot(dA[-1]*sigmoid_derivative(A[i+1]), self.W[i].T)
#             dW.append(dw_)
#             db.append(db_)
#             dA.append(dA_)
#         # reverse dW, db
#         dW = dW[::-1]
#         db = db[::-1]

#         # Gradient descent
#         for i in range(0, len(self.layers)-1):
#             self.W[i] = self.W[i] - self.alpha * dW[i]
#             self.b[i] = self.b[i] - self.alpha * db[i]

#     def fit(self, X, y, X_val, y_val, epochs=20, verbose=10):
#         for epoch in range(0, epochs):
#             self.fit_partial(X, y)
#             if epoch % verbose == 0:
#                 loss = self.calculate_loss(X, y)
#                 loss_val = self.calculate_loss(X_val, y_val)
#                 print("Epoch {}, loss {}, loss_val {}".format(epoch, loss, loss_val))

#     # Dự đoán
#     def predict(self, X):
#         for i in range(0, len(self.layers) - 1):
#             X = sigmoid(np.dot(X, self.W[i]) + (self.b[i].T))
#         return X

#     # Tính loss function
#     def calculate_loss(self, X, y):
#         y_predict = self.predict(X)
#         #return np.sum((y_predict-y)**2)/2
#         return -(np.sum(y*np.log(y_predict) + (1-y)*np.log(1-y_predict)))

In [ ]:
# p = NeuralNetwork([X_train.shape[1], 134, 1])
# p.fit(X_train_bow, y_train, X_val_bow, y_val, 20, 10)

In [ ]:
# # create neural network model
# odel = Sequential()
# model.add(Dense(64, input_dim=X_train_bow.shape[1], activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.fit(X_train_bow, y_train,
#           epochs=20,
#           batch_size=128)


# classifier with softmax

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# For a single-input model with 2 classes (binary classification):

model = Sequential()
model.add(Dense(70, activation='relu', input_dim=X_train_bow.shape[1]))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# bien save model
check_point = ModelCheckpoint('...input/quora-insincere-questions-classification/softmax_bow_ver_02.h5', monitor='val_loss', save_best_only=True)

class_weight = {0:1, 1:15}
# Train the model, iterating on the data in batches of 100 samples
model.fit(X_train_bow, y_train, epochs=10,validation_split=0.1, verbose=1, callbacks= [ check_point], batch_size=100, class_weight= class_weight)
print(model.summary())

In [ ]:
y_prod = model.predict_podba(X_val_bow)

In [ ]:
y_prod[y_prod >= 0.23].shape

In [ ]:
# choose with y_prod >= 0.23 => label = 1, y_prod < 0.23 => label = 0
y_class = np.squezze(y_prod >= 0.23).astype('int')

In [ ]:
print(classification_report(y_pred = y_class, y_true= y_val))